In [1]:
import boto3
import requests
import zipfile
import os
import pandas as pd
import json
import pymongo
import io

In [2]:
import subprocess

In [3]:
try:
  session = boto3.Session(region_name='ap-south-1',
                          aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
                          aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY'))
  print('Successfully Session Created')
except Exception as e:
  print(e)

Successfully Session Created


In [4]:
try:
  s3 = session.client(service_name='s3',region_name='ap-south-1')
  print('Successfully s3 client created')
except Exception as e:
  print(e)

Successfully s3 client created


In [5]:
for i in s3.list_buckets().get('Buckets'):
    print(i['Name'])

analytical-data-bucket
aws-glue-assets-894034347035-ap-south-1
bucket-athena-result
bucket-for-glue-script
bucket-for-lambda-function
cleaned-json-as-csv
raw-csv-from-jupyter
raw-csv-test123
raw-data-from-webpage
raw-json-from-jupyter
raw-json-test123


In [62]:
try:
    url = 'https://www.sec.gov/Archives/edgar/daily-index/bulkdata/submissions.zip'

    headers = {'user-agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.55'}

    response = requests.get(url,headers = headers,stream=True)

    content = response.content
    
    print('successfully got the zipfile from url')
    
except Exception as e:
    print(e)

successfully got the zipfile from url


In [62]:
try:
    url = 'https://www.sec.gov/Archives/edgar/daily-index/bulkdata/submissions.zip'

    headers = {'user-agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.55'}

    response = requests.get(url,headers = headers,stream=True)

    content = response.content
    
    print('successfully got the zipfile from url')
    
except Exception as e:
    print(e)

successfully got the zipfile from url


In [64]:
try:
    bucket_name = 'raw-data-from-webpage'
    file_path = 'largefile/file.zip'

    obj = s3.get_object(Bucket = bucket_name, Key = file_path)

    zip_file = io.BytesIO(obj['Body'].read())

    with zipfile.ZipFile(zip_file) as z:
      z.extractall()

    for file in z.namelist():
      s3.upload_file(Filename = file, Bucket = bucket_name, Key = 'unzipped/' + file)
    
    print('successfully extracted and uploaded in the unzipped folder')
    
except Exception as e:
    print(e)

successfully extracted and uploaded in the unzipped folder


In [23]:
bucket_name = 'raw-data-from-webpage'
folder_name = 'unzipped/'

for i in s3.list_objects(Bucket=bucket_name,Prefix=folder_name)['Contents']:
    print(i['Key'])

unzipped/CIK0000000003.json
unzipped/CIK0000000013.json
unzipped/CIK0000000014.json
unzipped/CIK0000000017.json
unzipped/CIK0000000018.json
unzipped/CIK0000000020.json
unzipped/CIK0000000049.json
unzipped/CIK0000000051.json
unzipped/CIK0000000063.json
unzipped/CIK0000001750-submissions-001.json
unzipped/CIK0000001750.json
unzipped/CIK0000001761.json
unzipped/CIK0000001800-submissions-001.json
unzipped/CIK0000001800-submissions-002.json
unzipped/CIK0000001800.json
unzipped/CIK0000001830.json
unzipped/CIK0000001841.json
unzipped/CIK0000001848.json
unzipped/CIK0000001853.json
unzipped/CIK0000001860.json
unzipped/CIK0000001904.json
unzipped/CIK0000001918.json
unzipped/CIK0000001923.json
unzipped/CIK0000001947.json
unzipped/CIK0000001952.json
unzipped/CIK0000001958.json
unzipped/CIK0000001961.json
unzipped/CIK0000001969.json
unzipped/CIK0000001985.json
unzipped/CIK0000001988.json
unzipped/CIK0000002024.json
unzipped/CIK0000002034-submissions-001.json
unzipped/CIK0000002034.json
unzipped/CIK

In [68]:
bucket_name = 'raw-data-from-webpage'
folder_name = 'unzipped/'

result = s3.list_objects(Bucket=bucket_name, Prefix=folder_name)

# get the number of files in the folder
if 'Contents' in result:
    file_count = len(result['Contents'])
else:
    file_count = 0

print(f'N.of files in {folder_name}: {file_count}')

N.of files in unzipped/: 1000


In [4]:
cmd = 'hdfs dfs -ls /'

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Directory content:')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

Directory content:
Found 4 items
drwxr-xr-x   - hdfs hdfsadmingroup          0 2023-03-30 11:38 /apps
drwxrwxrwt   - hdfs hdfsadmingroup          0 2023-03-30 11:40 /tmp
drwxr-xr-x   - hdfs hdfsadmingroup          0 2023-03-30 11:38 /user
drwxr-xr-x   - hdfs hdfsadmingroup          0 2023-03-30 11:38 /var



In [5]:
cmd = 'hdfs dfs -ls /user'

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Directory content:')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

Directory content:
Found 6 items
drwxrwxrwx   - hadoop hdfsadmingroup          0 2023-03-30 11:38 /user/hadoop
drwxr-xr-x   - mapred mapred                  0 2023-03-30 11:38 /user/history
drwxrwxrwx   - hdfs   hdfsadmingroup          0 2023-03-30 11:40 /user/hive
drwxrwxrwx   - livy   livy                    0 2023-03-30 11:38 /user/livy
drwxrwxrwx   - root   hdfsadmingroup          0 2023-03-30 11:38 /user/root
drwxrwxrwx   - spark  spark                   0 2023-03-30 11:38 /user/spark



In [19]:
cmd = 'hdfs dfs -mkdir /user/hadoop/sec_data'

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Succesfully directory created.')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

Succesfully directory created.


In [15]:
cmd = 'hdfs dfs -ls /user/hadoop'

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Directory content:')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

Directory content:
Found 1 items
drwxr-xr-x   - emr-notebook hdfsadmingroup          0 2023-03-30 12:03 /user/hadoop/sec_data



In [16]:
cmd = 'hadoop fs -cp s3://raw-data-from-webpage/Unzipped/* /user/hadoop/sec_data'

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Successfully copied to hdfs')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

Successfully copied to hdfs



In [18]:
cmd = 'hdfs dfs -ls /user/hadoop/sec_data'

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Directory content:')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

Directory content:
Found 34 items
-rw-r--r--   1 emr-notebook hdfsadmingroup     148469 2023-03-30 12:08 /user/hadoop/sec_data/CIK0000001750-submissions-001.json
-rw-r--r--   1 emr-notebook hdfsadmingroup     272089 2023-03-30 12:08 /user/hadoop/sec_data/CIK0000001800-submissions-001.json
-rw-r--r--   1 emr-notebook hdfsadmingroup      29781 2023-03-30 12:08 /user/hadoop/sec_data/CIK0000001800-submissions-002.json
-rw-r--r--   1 emr-notebook hdfsadmingroup      23668 2023-03-30 12:08 /user/hadoop/sec_data/CIK0000002034-submissions-001.json
-rw-r--r--   1 emr-notebook hdfsadmingroup       7810 2023-03-30 12:08 /user/hadoop/sec_data/CIK0000002135-submissions-001.json
-rw-r--r--   1 emr-notebook hdfsadmingroup     268274 2023-03-30 12:08 /user/hadoop/sec_data/CIK0000002488-submissions-001.json
-rw-r--r--   1 emr-notebook hdfsadmingroup     213991 2023-03-30 12:08 /user/hadoop/sec_data/CIK0000002969-submissions-001.json
-rw-r--r--   1 emr-notebook hdfsadmingroup      31584 2023-03-30 12:08

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Hadoop_Project') \
                            .enableHiveSupport() \
                            .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
                            .getOrCreate()
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1680157389837_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
file_path = "hdfs://ec2-65-0-45-11.ap-south-1.compute.amazonaws.com/user/hadoop/sec_data/CIK0000001750-submissions-001.json"
df = spark.read.json(file_path)

df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+
|  acceptanceDateTime|     accessionNumber|                 act|          fileNumber|          filingDate|          filmNumber|                form|        isInlineXBRL|              isXBRL|               items|primaryDocDescription|     primaryDocument|          reportDate|                size|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+
|[2010-04-26T11:52...|[0001104659-10-02...|[, 34, , , 34, , ...|[, 001-06263, , ,...|[2010-04-26, 2010...|[, 

In [4]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- acceptanceDateTime: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- accessionNumber: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- act: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fileNumber: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filingDate: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filmNumber: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- form: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isInlineXBRL: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- isXBRL: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- primaryDocDescription: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- primaryDocu

In [3]:
df1 = df.select('acceptanceDateTime') \
        .withColumn('acceptanceDateTime', explode('acceptanceDateTime')) \
        .withColumn("id", monotonically_increasing_id())
df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+
|  acceptanceDateTime| id|
+--------------------+---+
|2010-04-26T11:52:...|  0|
|2010-04-12T17:15:...|  1|
|2010-04-07T16:42:...|  2|
|2010-04-02T12:13:...|  3|
|2010-03-29T17:24:...|  4|
|2010-03-22T10:04:...|  5|
|2010-03-19T15:55:...|  6|
|2010-03-18T16:23:...|  7|
|2010-03-17T06:01:...|  8|
|2010-03-05T20:35:...|  9|
|2010-03-05T15:33:...| 10|
|2010-03-03T14:06:...| 11|
|2010-03-03T13:43:...| 12|
|2010-03-03T13:27:...| 13|
|2010-02-12T17:20:...| 14|
|2010-02-12T13:03:...| 15|
|2010-02-10T06:17:...| 16|
|2010-02-09T17:13:...| 17|
|2010-01-29T14:12:...| 18|
|2010-01-28T14:50:...| 19|
+--------------------+---+
only showing top 20 rows

In [23]:
type(df1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [4]:
df2 = df.select('accessionNumber').withColumn('accessionNumber', explode('accessionNumber')).withColumn("id", monotonically_increasing_id())
df3 = df.select('act').withColumn('act', explode('act')).withColumn("id", monotonically_increasing_id())
df4 = df.select('fileNumber').withColumn('fileNumber', explode('fileNumber')).withColumn("id", monotonically_increasing_id())
df5 = df.select('filingDate').withColumn('filingDate', explode('filingDate')).withColumn("id", monotonically_increasing_id())
df6 = df.select('filmNumber').withColumn('filmNumber', explode('filmNumber')).withColumn("id", monotonically_increasing_id())
df7 = df.select('form').withColumn('form', explode('form')).withColumn("id", monotonically_increasing_id())
df8 = df.select('isInlineXBRL').withColumn('isInlineXBRL', explode('isInlineXBRL')).withColumn("id", monotonically_increasing_id())
df9 = df.select('isXBRL').withColumn('isXBRL', explode('isXBRL')).withColumn("id", monotonically_increasing_id())
df10 = df.select('items').withColumn('items', explode('items')).withColumn("id", monotonically_increasing_id())
df11 = df.select('primaryDocDescription').withColumn('primaryDocDescription', explode('primaryDocDescription')).withColumn("id", monotonically_increasing_id())
df12 = df.select('primaryDocument').withColumn('primaryDocument', explode('primaryDocument')).withColumn("id", monotonically_increasing_id())
df13 = df.select('reportDate').withColumn('reportDate', explode('reportDate')).withColumn("id", monotonically_increasing_id())
df14 = df.select('size').withColumn('size', explode('size')).withColumn("id", monotonically_increasing_id())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_join = df1.join(df2, "id").join(df3, "id").join(df4, "id").join(df5, "id") \
          .join(df6, "id").join(df7, "id").join(df8, "id").join(df9, "id") \
          .join(df10, "id").join(df11, "id").join(df12, "id").join(df13, "id") \
          .join(df14, "id").drop("id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
type(df_join)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [7]:
df_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---+----------+----------+----------+--------+------------+------+--------------+---------------------+--------------------+----------+-------+
|  acceptanceDateTime|     accessionNumber|act|fileNumber|filingDate|filmNumber|    form|isInlineXBRL|isXBRL|         items|primaryDocDescription|     primaryDocument|reportDate|   size|
+--------------------+--------------------+---+----------+----------+----------+--------+------------+------+--------------+---------------------+--------------------+----------+-------+
|2010-04-26T11:52:...|0001104659-10-021700|   |          |2010-04-26|          |SC 13D/A|           0|     0|              |             SC 13D/A|a10-8551_1sc13da.htm|          | 128457|
|2010-04-12T17:15:...|0001104659-10-019368| 34| 001-06263|2010-04-12|  10745530|     8-K|           0|     0|1.01,2.01,9.01|                  8-K|    a10-8035_18k.htm|2010-04-07|  58759|
|2010-04-07T16:42:...|0001127602-10-010769|   |          |2010-04

In [8]:
type(df_join)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [9]:
spark.sql("SHOW DATABASES").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|namespace|
+---------+
|  default|
+---------+

In [10]:
spark.sql("SHOW tables").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

In [11]:
df_join.createOrReplaceTempView("temp_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark.sql("SHOW tables").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|        |temp_table|       true|
+--------+----------+-----------+

In [12]:
spark.sql("CREATE TABLE sec_table AS SELECT * FROM temp_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [13]:
spark.sql("SHOW tables").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
| default| sec_table|      false|
|        |temp_table|       true|
+--------+----------+-----------+

In [35]:
spark.sql("DESC sec_table").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|  acceptanceDateTime|   string|   null|
|     accessionNumber|   string|   null|
|                 act|   string|   null|
|          fileNumber|   string|   null|
|          filingDate|   string|   null|
|          filmNumber|   string|   null|
|                form|   string|   null|
|        isInlineXBRL|   bigint|   null|
|              isXBRL|   bigint|   null|
|               items|   string|   null|
|primaryDocDescrip...|   string|   null|
|     primaryDocument|   string|   null|
|          reportDate|   string|   null|
|                size|   bigint|   null|
+--------------------+---------+-------+

In [14]:
df_join.write.mode("append").insertInto("sec_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
spark.sql("SELECT * FROM sec_table LIMIT 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---+----------+----------+----------+--------+------------+------+--------------+---------------------+--------------------+----------+-------+
|  acceptanceDateTime|     accessionNumber|act|fileNumber|filingDate|filmNumber|    form|isInlineXBRL|isXBRL|         items|primaryDocDescription|     primaryDocument|reportDate|   size|
+--------------------+--------------------+---+----------+----------+----------+--------+------------+------+--------------+---------------------+--------------------+----------+-------+
|2010-04-26T11:52:...|0001104659-10-021700|   |          |2010-04-26|          |SC 13D/A|           0|     0|              |             SC 13D/A|a10-8551_1sc13da.htm|          | 128457|
|2010-04-12T17:15:...|0001104659-10-019368| 34| 001-06263|2010-04-12|  10745530|     8-K|           0|     0|1.01,2.01,9.01|                  8-K|    a10-8035_18k.htm|2010-04-07|  58759|
|2010-04-07T16:42:...|0001127602-10-010769|   |          |2010-04

In [16]:
spark.sql("SELECT COUNT(*) FROM sec_table LIMIT 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|    1982|
+--------+

In [24]:
dir_path = "hdfs://ec2-65-0-45-11.ap-south-1.compute.amazonaws.com/user/hadoop/sec_data/"

# List file names in HDFS directory
proc = subprocess.Popen(["hadoop", "fs", "-ls", dir_path], stdout=subprocess.PIPE)
out, err = proc.communicate()

# Convert output to list of file names
file_names = [line.split()[-1].split('/')[-1] for line in out.decode().strip().split('\n')[1:]]
print(file_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['CIK0000001750-submissions-001.json', 'CIK0000001800-submissions-001.json', 'CIK0000001800-submissions-002.json', 'CIK0000002034-submissions-001.json', 'CIK0000002135-submissions-001.json', 'CIK0000002488-submissions-001.json', 'CIK0000002969-submissions-001.json', 'CIK0000003116-submissions-001.json', 'CIK0000003197-submissions-001.json', 'CIK0000003370-submissions-001.json', 'CIK0000003453-submissions-001.json', 'CIK0000003521-submissions-001.json', 'CIK0000003545-submissions-001.json', 'CIK0000003570-submissions-001.json', 'CIK0000003673-submissions-001.json', 'CIK0000003794-submissions-001.json', 'CIK0000003906-submissions-001.json', 'CIK0000004127-submissions-001.json', 'CIK0000004281-submissions-002.json', 'CIK0000004447-submissions-001.json', 'CIK0000004515-submissions-001.json', 'CIK0000004904-submissions-001.json', 'CIK0000004962-submissions-001.json', 'CIK0000004977-submissions-001.json', 'CIK0000005133-submissions-001.json', 'CIK0000005272-submissions-001.json', 'CIK0000005

In [25]:
for filename in file_names:
    
    file_path = f"hdfs://ec2-65-0-45-11.ap-south-1.compute.amazonaws.com/user/hadoop/sec_data/{filename}"
    
    df = spark.read.json(file_path)
    
    df1 = df.select('acceptanceDateTime').withColumn('acceptanceDateTime', explode('acceptanceDateTime')).withColumn("id", monotonically_increasing_id())
    df2 = df.select('accessionNumber').withColumn('accessionNumber', explode('accessionNumber')).withColumn("id", monotonically_increasing_id())
    df3 = df.select('act').withColumn('act', explode('act')).withColumn("id", monotonically_increasing_id())
    df4 = df.select('fileNumber').withColumn('fileNumber', explode('fileNumber')).withColumn("id", monotonically_increasing_id())
    df5 = df.select('filingDate').withColumn('filingDate', explode('filingDate')).withColumn("id", monotonically_increasing_id())
    df6 = df.select('filmNumber').withColumn('filmNumber', explode('filmNumber')).withColumn("id", monotonically_increasing_id())
    df7 = df.select('form').withColumn('form', explode('form')).withColumn("id", monotonically_increasing_id())
    df8 = df.select('isInlineXBRL').withColumn('isInlineXBRL', explode('isInlineXBRL')).withColumn("id", monotonically_increasing_id())
    df9 = df.select('isXBRL').withColumn('isXBRL', explode('isXBRL')).withColumn("id", monotonically_increasing_id())
    df10 = df.select('items').withColumn('items', explode('items')).withColumn("id", monotonically_increasing_id())
    df11 = df.select('primaryDocDescription').withColumn('primaryDocDescription', explode('primaryDocDescription')).withColumn("id", monotonically_increasing_id())
    df12 = df.select('primaryDocument').withColumn('primaryDocument', explode('primaryDocument')).withColumn("id", monotonically_increasing_id())
    df13 = df.select('reportDate').withColumn('reportDate', explode('reportDate')).withColumn("id", monotonically_increasing_id())
    df14 = df.select('size').withColumn('size', explode('size')).withColumn("id", monotonically_increasing_id())
    
    df_join = df1.join(df2, "id").join(df3, "id").join(df4, "id").join(df5, "id") \
              .join(df6, "id").join(df7, "id").join(df8, "id").join(df9, "id") \
              .join(df10, "id").join(df11, "id").join(df12, "id").join(df13, "id") \
              .join(df14, "id").drop("id")
    
    df_join.write.mode("append").insertInto("sec_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
spark.sql("SELECT COUNT(*) FROM sec_table LIMIT 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|   33329|
+--------+

In [27]:
spark.sql("SELECT * FROM sec_table LIMIT 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---+----------+----------+----------+----+------------+------+-----+---------------------+--------------------+----------+------+
|  acceptanceDateTime|     accessionNumber|act|fileNumber|filingDate|filmNumber|form|isInlineXBRL|isXBRL|items|primaryDocDescription|     primaryDocument|reportDate|  size|
+--------------------+--------------------+---+----------+----------+----------+----+------------+------+-----+---------------------+--------------------+----------+------+
|2012-02-28T15:07:...|0001104659-12-013978| 33| 033-04959|2012-02-28|  12646582|497K|           0|     0|     |                 497K|a11-32272_23497k.htm|          |472927|
|2012-02-28T15:06:...|0001104659-12-013976| 33| 033-04959|2012-02-28|  12646577|497K|           0|     0|     |                 497K|a11-32272_21497k.htm|          |504520|
|2012-02-28T15:06:...|0001104659-12-013975| 33| 033-04959|2012-02-28|  12646575|497K|           0|     0|     |                 497K|a1

In [23]:
cmd = "hive -e 'SHOW DATABASES;'"

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Databases:')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Databases:
default

In [24]:
cmd = "hive -e 'SELECT * FROM sec_table LIMIT 5;'"

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print('Directory content:')
    print(result.stdout)
else:
    print('Error accessing directory')
    print(result.stderr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Directory content:
2010-04-26T11:52:29.000Z	0001104659-10-021700			2010-04-26		SC 13D/A	0	0		SC 13D/A	a10-8551_1sc13da.htm		128457
2010-04-12T17:15:56.000Z	0001104659-10-019368	34	001-06263	2010-04-12	10745530	8-K	0	0	1.01,2.01,9.01	8-K	a10-8035_18k.htm	2010-04-07	58759
2010-04-07T16:42:24.000Z	0001127602-10-010769			2010-04-07		4	0	0		PRIMARY DOCUMENT	xslF345X03/form4.xml	2010-04-06	8078
2010-04-02T12:13:19.000Z	0001127602-10-010179			2010-04-02		4	0	0		PRIMARY DOCUMENT	xslF345X03/form4.xml	2010-04-01	12176
2010-03-29T17:24:20.000Z	0001104659-10-017022	34	001-06263	2010-03-29	10711495	8-K	0	0	1.01,8.01,9.01	8-K	a10-7157_18k.htm	2010-03-25	855058